In [1]:
# Sequential Capacitated Facility Location Problem (S-CFLP) Solver using CPLEX
# This script implements a bilevel optimization structure for the S-CFLP.
# The leader (upper level) selects up to p facilities to open (x_j variables).
# The follower (lower level, competitor) then selects up to r facilities to open (y_j variables) 
# in response, to maximize its own objective (which reduces the leader's market share).
# Customers then probabilistically choose between leader and competitor based on offered facilities.
# We use IBM CPLEX (docplex) for modeling and solving. Here, we demonstrate a straightforward 
# approach by enumerating the follower's best response for each leader decision (suitable for small instances).
# In practice, one could use more advanced reformulation or cutting-plane methods to handle larger instances.

from itertools import combinations
from math import inf

# ----- 1. Define the problem instance parameters -----
# You can modify these parameters or load them from a data file for different instances.
# Example instance size |I|-|J|-p-r = 5-5-2-2 (5 customers, 5 potential facility sites, p=2, r=2).
nI = 5  # number of customers
nJ = 5  # number of facility locations (sites)

p = 2   # leader's budget (max number of facilities the leader can open)
r = 2   # follower's budget (max number of facilities the follower can open)

# Customer demand weights h_i (e.g., population or importance of customer i).
# For simplicity, we use random or fixed values. Here we assign h_i = 1 for all i (equal weight).
h = [1 for i in range(nI)]

# Utility parameters U^L_i and U^F_i for each customer i (baseline utility for leader and follower).
# These represent inherent preference or loyalty towards the leader or competitor, even if no facility is opened.
# We will assign some example values (could also be loaded or set differently per instance).
# U_L = [10 for i in range(nI)]  # baseline utility for leader (e.g., brand loyalty)
# U_F = [10 for i in range(nI)]  # baseline utility for follower/competitor
U_L = [0 for i in range(nI)]
U_F = [0 for i in range(nI)] 
print(U_L)

[0, 0, 0, 0, 0]


In [2]:

# Customer-facility preference weights w_{ij} for each customer i and facility j.
# These indicate the additional utility a customer i gains if facility j is available to them.
# We generate an example matrix of weights (e.g., based on proximity or service quality).
import random
random.seed(0)
w = [[random.randint(1, 10) for j in range(nJ)] for i in range(nI)]
# (In a real scenario, w_{ij} could be input data. Each row i corresponds to customer i's utility gain from each facility j.)

# ----- 2. Define functions for objective calculations -----
def leader_objective(x_set, y_set):
    """Compute the leader's objective value L^+(x, y) for given open facility sets:
    x_set: set of facilities opened by leader (indices of sites),
    y_set: set of facilities opened by follower (competitor).
    Returns the total leader utility (market share) L^+."""
    total_leader_share = 0.0
    for i in range(nI):
        # Calculate the total "attraction" for leader and competitor for customer i
        A_i = U_L[i] + sum(w[i][j] for j in x_set)              # leader's total utility for customer i
        B_i = U_F[i] + sum(w[i][j] for j in y_set)              # competitor's total utility for customer i
        if A_i + B_i == 0:
            # Avoid division by zero (if both A_i and B_i are zero, which is unlikely if baseline utilities are positive)
            share_i = 0
        else:
            share_i = A_i / (A_i + B_i)                        # fraction of customer i's demand that goes to leader
        total_leader_share += h[i] * share_i                   # weighted by customer i's demand h_i
    return total_leader_share

def competitor_objective(x_set, y_set):
    """Compute the follower's (competitor's) objective value for given leader and follower facility sets.
    This is the total competitor utility (market share) for the follower."""
    total_comp_share = 0.0
    for i in range(nI):
        A_i = U_L[i] + sum(w[i][j] for j in x_set)
        B_i = U_F[i] + sum(w[i][j] for j in y_set)
        if A_i + B_i == 0:
            share_i = 0
        else:
            share_i = B_i / (A_i + B_i)                        # competitor's share fraction for customer i
        total_comp_share += h[i] * share_i
    return total_comp_share

def best_response_of_competitor(x_set):
    """Given a set of leader-opened facilities (x_set), determine the competitor's best response:
    i.e., the set of up to r facilities y_set (disjoint from x_set) that maximizes the competitor's objective.
    Returns a tuple (best_y_set, best_comp_obj)."""
    best_y_set = set()
    best_obj = -inf
    # All possible subsets of sites (from J \ x_set) of size <= r (competitor can open at most r facilities)
    available_sites = [j for j in range(nJ) if j not in x_set]
    # We consider all combinations of allowable sites for the competitor (from size 0 up to r)
    # This brute-force approach is feasible for small instances. For larger instances, a specialized algorithm or MILP reformulation is needed.
    for k in range(r + 1):  # k = number of facilities competitor opens (0 to r)
        for combo in combinations(available_sites, k):
            y_set = set(combo)
            # Calculate competitor's objective for this choice
            comp_obj = competitor_objective(x_set, y_set)
            if comp_obj > best_obj:
                best_obj = comp_obj
                best_y_set = y_set
    return best_y_set, best_obj

# ----- 3. Solve the upper-level problem (leader's optimization anticipating competitor) -----
# We will compute the optimal leader decision by considering each possible set of p facilities for the leader,
# and evaluating the resulting objective L^+ given the competitor's best response.
# This approach explicitly searches the space of leader decisions (combinatorial search), suitable when p and nJ are small.
# For larger problems, one could integrate this search with the CPLEX solver using cutting planes or KKT-based reformulation.

best_leader_obj = -inf
best_x_set = None
best_y_set = None

# Enumerate all combinations of p facilities out of nJ for the leader
for combo in combinations(range(nJ), p):
    x_set = set(combo)
    # Given this leader decision x_set, compute competitor's best response
    y_set, comp_obj = best_response_of_competitor(x_set)
    # Now compute the leader's resulting objective (market share) L^+ for this scenario
    leader_obj_val = leader_objective(x_set, y_set)
    # Check if this is the best leader outcome found so far
    if leader_obj_val > best_leader_obj:
        best_leader_obj = leader_obj_val
        best_x_set = x_set
        best_y_set = y_set

# ----- 4. Output the results -----
print("Optimal leader facility choices (x_j = 1):", sorted(best_x_set))
print("Optimal follower (competitor) facility choices in response (y_j = 1):", sorted(best_y_set))
print(f"Leader's objective value L^+(x,y) = {best_leader_obj:.3f}")
# We can also output the competitor's objective for completeness (this is the follower's achieved market share)
print(f"Competitor's objective value = {competitor_objective(best_x_set, best_y_set):.3f}")

# Each element in best_x_set or best_y_set is an index of the facility location chosen by leader or competitor respectively.
# The leader's objective value is the total weighted customer share captured by the leader's facilities.
# The competitor's objective value is the total weighted share captured by competitor's facilities (the complement of leader's share).


Optimal leader facility choices (x_j = 1): [0, 4]
Optimal follower (competitor) facility choices in response (y_j = 1): [1, 2]
Leader's objective value L^+(x,y) = 2.841
Competitor's objective value = 2.159
